In [ ]:
# Importing necessary libraries
import geopandas as gpd
import rasterio.mask
import fiona
import os
import numpy as np
import shapefile
import rasterstats
import matplotlib.pyplot as plt
import statistics
import pandas as pd
import rasterio
import time
import cv2
import math

In [ ]:
from typing import cast, Optional

# Importing necessary programs from the libraries
from rasterio.plot import show
from rasterio.transform import from_origin
from rasterio.crs import CRS
from rasterio import features
from rasterstats import zonal_stats
from matplotlib.colors import ListedColormap
from matplotlib import pyplot as plt

In [ ]:
import PIL.Image

def img_scale(img: np.ndarray) -> np.ndarray:
    return (img * 255).astype(np.uint8)

def display_image(img: np.ndarray, _: Optional[str] = None):
    display(PIL.Image.fromarray(img))

def display_contours(img: np.ndarray, contours):
    img_with_contours = img.copy()
    for contour in contours:
        cv2.drawContours(img_with_contours, [contour], 0, (0, 255, 0), 2)
    display_image(img_with_contours)

In [ ]:
img = cv2.imread('Test_Images/temp_ndvi.jpg')
img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
col_grey_img = cv2.cvtColor(img_grey, cv2.COLOR_GRAY2RGB) # for printing on top of
display_image(img_grey)

In [ ]:
initial_contours, _ = cv2.findContours(img_grey, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

initial_contour_mask = np.zeros(img_grey.shape, np.uint8)

for contour in initial_contours:
    if cv2.contourArea(contour) < 600: continue

    cv2.drawContours(initial_contour_mask, [contour], 0, (255,), -1)

display_image(initial_contour_mask)

In [ ]:
img_large_contours = cv2.bitwise_or(img_grey, img_grey, mask=initial_contour_mask)

display_image(img_large_contours)

In [ ]:
img_contour_eroded = cv2.erode(
    np.where(cast(np.ndarray, img_large_contours) > 0, [255], [0]).astype(np.uint8),
    img_large_contours,
    np.ones((3, 3), np.uint8),
    iterations = 1
)

display_image(img_contour_eroded)

In [ ]:
img_eroded_dilated = cv2.dilate(
    img_contour_eroded,
    np.ones((5,5), np.uint8),
    iterations = 1
)

display_image(img_eroded_dilated)

In [ ]:
segment_detector = cv2.createLineSegmentDetector()
lines, *_ = segment_detector.detect(img_eroded_dilated)

lines = lines[np.squeeze((0 < cast(np.ndarray, lines)).all(axis=2))]

segment_image = np.copy(col_grey_img)
for line in lines:
    x1, y1, x2, y2 = np.rint(line.flatten()).astype(np.int32)
    cv2.line(segment_image, (x1, y1), (x2, y2), (255, 0, 0), 2)

print(len(lines))
display_image(segment_image)

In [ ]:
img_contour_edges = cv2.Canny(img_eroded_dilated, 30, 200)

display_image(img_contour_edges)

In [ ]:
rho = 2  # distance resolution in pixels of the Hough grid
theta = np.pi / 180  # angular resolution in radians of the Hough grid
threshold = 15  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 100  # minimum number of pixels making up a line
max_line_gap = 40  # maximum gap in pixels between connectable line segments
line_image = np.copy(col_grey_img)

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
lines = cv2.HoughLinesP(img_contour_edges, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)

for line in lines:
    for x1, y1, x2, y2 in line:
        cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 2)

print(len(lines))
print(sum(len(line) for line in lines))
display_image(line_image)